## Dependencies 

In [1]:
%pip install pyaudio SpeechRecognition keyboard

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyaudio
import keyboard

In [3]:
%pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pydub SpeechRecognition word2number


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import re
from word2number import w2n
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
import json
from IPython.display import display, Javascript
import base64
import speech_recognition as sr
from pydub import AudioSegment
import io

# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


c:\Users\dxl33\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dxl33\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dxl33\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Define Recorder Class

In [7]:
class Recorder:
    def __init__(self, silence_timeout=15):
        self.recognizer = sr.Recognizer()
        self.silence_timeout = silence_timeout  # Time to wait for silence before stopping

    def record_audio(self):
        with sr.Microphone() as source:
            print("Adjusting for ambient noise, please wait...")
            self.recognizer.adjust_for_ambient_noise(source)
            print("Recording... Speak now.")
            
            # Record audio until silence is detected
            audio_data = self.recognizer.listen(source, timeout=self.silence_timeout, phrase_time_limit=self.silence_timeout)
            print("Recording stopped.")
            
            return audio_data


## Define Transcriber Class

In [8]:
class Transcriber:
    def __init__(self):
        self.recognizer = sr.Recognizer()
    
    def transcribe_audio(self, audio_data):
        try:
            print("Transcribing...")
            prompt = self.recognizer.recognize_google(audio_data)
            print("You said: " + prompt)
            return prompt
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return ""

## Record Audio Prompt

Skip if you are Typing a Prompt in Next Section

In [40]:
def main():
    # Create a Recorder instance with a silence timeout (e.g., 5 seconds)
    recorder = Recorder(silence_timeout=15)
    transcriber = Transcriber()

    audio_data = recorder.record_audio()
    prompt = transcriber.transcribe_audio(audio_data)
    
    # Check if prompt is empty
    if not prompt:
        print("No transcription available.")
    else:
        print(f"Transcribed Prompt: {prompt}")

    # 'prompt' is now defined and can be used in further processing
    return prompt

# Run the main function and capture the prompt
prompt = main()


Adjusting for ambient noise, please wait...
Recording... Speak now.
Recording stopped.
Transcribing...
You said: I like cheeseburgers you're the best especially with bacon love them
Transcribed Prompt: I like cheeseburgers you're the best especially with bacon love them


## Type Prompt

Skip if you Recorded a Prompt in Previous Section

In [62]:
#Option to Type Prompt Instead
prompt = "I need to update tables 5 11 and 17 to have 3.25 foot sides and a height of 3 feet and use a leg insert of 1.25 inches make it 0.75 inches thick with corner fillets of 5.25 inches."

## Find / Assign Variables

In [36]:
import re
import json

### Create or Modify

In [65]:
# Define keywords for actions
action_keywords = {
    "Create": ["create", "make", "build", "design", "generate"],
    "Modify": ["modify", "change", "adjust", "alter", "update"]
}

# Function to detect action
def detect_action(text):
    # Convert text to lowercase for case-insensitive comparison
    text = text.lower()
    print(f"Processing text: '{text}'")  # Debugging: print the processed text

    # Initialize action variable
    detected_action = "UNKNOWN"

    # Check for CREATE keywords
    create_keywords = action_keywords["Create"]
    if any(keyword in text for keyword in create_keywords):
        detected_action = "Create"

    # Check for MODIFY keywords
    modify_keywords = action_keywords["Modify"]
    if any(keyword in text for keyword in modify_keywords):
        detected_action = "Modify"

    return detected_action

# Use the prompt to determine action
action = detect_action(prompt)

print(f"Detected Action: {action}")


Processing text: 'i need to update tables 5 11 and 17 to have 3.25 foot sides and a height of 3 feet and use a leg insert of 1.25 inches make it 0.75 inches thick with corner fillets of 5.25 inches.'
Detected Action: Modify


### Variables

In [66]:
# Define possible synonyms for each parameter
parameter_synonyms = {
    "Height": ["height", "tall", "elevation", "high"],
    "Corner Fillet": ["corner fillet", "fillet", "rounded corner", "corner fillets", "fillets"],
    "Leg Insert": ["leg insert", "leg distance", "leg spacing", "inserts", "leg insert distance of "],
    "Sides": ["sides", "edges"],
    "Thickness": ["thickness", "thick", "depth"]
}

# Function to find the closest number to a keyword, considering proximity and grammar
def find_closest_value(keyword, text):
    pattern = re.compile(r'(\b\d+(\.\d+)?\b)\s*(feet|foot|inches|inch|ft|in|inches.|sides)?')
    matches = pattern.finditer(text)
    keyword_position = text.find(keyword)

    closest_distance = float('inf')
    closest_value = None

    for match in matches:
        number, _, unit = match.groups()
        start, end = match.span()

        # Calculate the distance between the keyword and the number
        distance = abs(keyword_position - start)

        # Determine the direction (before or after) of the keyword relative to the number
        if keyword_position < start:  # Keyword before number
            direction = "after"
        else:  # Keyword after number
            direction = "before"

        # Only consider this match if it's closer than any previously found
        if distance < closest_distance:
            closest_distance = distance
            closest_value = f"{number} {unit}".strip()

            # Ensure grammatical correctness by checking direction
            if direction == "after" and closest_distance > len(keyword):
                continue

    return closest_value

# Extract parameters
parameters = {}

for parameter, synonyms in parameter_synonyms.items():
    for synonym in synonyms:
        if synonym in prompt:
            value = find_closest_value(synonym, prompt)
            if value:
                parameters[parameter] = value
                break



print(json.dumps(parameters, indent=2))

{
  "Height": "3 feet",
  "Corner Fillet": "5.25 inches",
  "Leg Insert": "1.25 inches",
  "Sides": "3.25 foot",
  "Thickness": "0.75 inches"
}


## ID Extraction for Modified Elements 

In [67]:
action

'Modify'

In [68]:
def extract_ids(text):
    # Normalize the text to lowercase
    text = text.lower()

    # Define keywords indicating ID context
    id_keywords = ['table', 'tables']

    # Initialize a list to store IDs
    ids = []

    # Split the text into words
    words = text.split()

    # Iterate through the words to find IDs after the keywords
    for i, word in enumerate(words):
        if word in id_keywords:
            # Collect numbers that follow the keyword, handling cases with "and"
            j = i + 1
            while j < len(words):
                if words[j].isdigit():
                    ids.append(int(words[j]))
                elif 'and' in words[j]:
                    # Continue if "and" is present to capture the number after it
                    j += 1
                    if j < len(words) and words[j].isdigit():
                        ids.append(int(words[j]))
                elif ',' in words[j]:
                    # Handle cases where numbers are followed by a comma
                    comma_separated_ids = [int(num) for num in words[j].split(',') if num.isdigit()]
                    ids.extend(comma_separated_ids)
                else:
                    break
                j += 1

    # Remove duplicates and sort IDs
    ids = sorted(set(ids))

    return ids

# Test the function with the example prompt
ids = extract_ids(prompt)
print(f"Modified IDs: {ids}")



Modified IDs: [5, 11, 17]


In [69]:
print(action)
print(ids)
print(parameters)

Modify
[5, 11, 17]
{'Height': '3 feet', 'Corner Fillet': '5.25 inches', 'Leg Insert': '1.25 inches', 'Sides': '3.25 foot', 'Thickness': '0.75 inches'}


## Prepare / Export Revit Library

In [45]:
# Function to extract just the numeric part of a value
def extract_number(value):
    # Regular expression to find the numeric part of the string
    match = re.search(r'\d+(\.\d+)?', value)
    if match:
        return match.group()
    return "Not specified"

# Prepare Revit parameters including action type
def prepare_revit_parameters(parameters, action, ids):
    # Define the parameter dictionary with mappings to Revit family parameters
    revit_parameters = {
        "Action": action,
        "Table IDs": ids,
        "Parameters": {
            "Height": {
                "ParameterName": "Height",
                "Value": extract_number(parameters.get("Height", "Not specified")),
                "Unit": "feet"  # You can adjust or remove this if needed
            },
            "Thickness": {
                "ParameterName": "Thickness",
                "Value": extract_number(parameters.get("Thickness", "Not specified")),
                "Unit": "inches"
            },
            "Sides": {
                "ParameterName": "Number of Sides",
                "Value": extract_number(parameters.get("Sides", "Not specified")),
                "Unit": "feet"  # You can adjust or remove this if needed
            },
            "Leg Insert": {
                "ParameterName": "Leg Insert",
                "Value": extract_number(parameters.get("Leg Insert", "Not specified")),
                "Unit": "inches"
            },
            "Corner Fillet": {
                "ParameterName": "Corner Fillet",
                "Value": extract_number(parameters.get("Corner Fillet", "Not specified")),
                "Unit": "inches"
            }
        }
    }

    return revit_parameters

# Prepare Revit parameters
revit_parameters = prepare_revit_parameters(parameters, action, ids)

# Print the result
print(json.dumps(revit_parameters, indent=4))

{
    "Action": "Create",
    "Table IDs": [],
    "Parameters": {
        "Height": {
            "ParameterName": "Height",
            "Value": "3",
            "Unit": "feet"
        },
        "Thickness": {
            "ParameterName": "Thickness",
            "Value": "0.75",
            "Unit": "inches"
        },
        "Sides": {
            "ParameterName": "Number of Sides",
            "Value": "3.25",
            "Unit": "feet"
        },
        "Leg Insert": {
            "ParameterName": "Leg Insert",
            "Value": "1.25",
            "Unit": "inches"
        },
        "Corner Fillet": {
            "ParameterName": "Corner Fillet",
            "Value": "5.25",
            "Unit": "inches"
        }
    }
}


In [34]:
# Define the filename for the JSON file
json_filename = "Test2Parameter.json"

In [93]:
# Write the dictionary to a JSON file - Writes dictionary to same folder as notebook
with open(json_filename, 'w') as json_file:
    json.dump(revit_parameters, json_file, indent=4)